# 10 Health Model

**Project:** NORI  
**Author:** Yuseof J  
**Date:** 23/12/25

### **Purpose**
Load all  features and health outcomes (model targets), train different models and evaluate performance, then select the best performing model to make final health outcome predictions across all NYC tracts. 

What this model is doing exactly:

This model is learning correlative relationships between neighborhood demographics, the built environment, and tract-level health outcomes observed in the population.

**Important limitation**: Health outcomes are heavily influenced by personal choices and lifestyle. Though demopgraphics and the environment also impact health, it is expected that the current feature set will only capture a small (but meaningful) portion of health outcome dynamics in relation to neighborhood-level characteristics. 

### **Inputs**
- `data/processed/master_features.csv`
- `data/processed/outcomes_health.csv`

### **Outputs**
- `models/xgb_health_model.pkl`
- `data/processed/predictions_health.csv`
- `data/processed/model_performance_health.csv`
- `data/processed/nyc_tracts.gpkg (layer = health_predictions)`
--------------------------------------------------------------------------

NOTE: This first pass is really just meant to get the ML and SHAP scaffolding set up. As of now, not enough model features are available to expect great model performance. 

### 0. Imports and Setup

In [1]:
# package imports
import os
import joblib
import numpy as np
import pandas as pd
import geopandas as gpd
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.model_selection import cross_val_score, GroupKFold

# specify filepaths
path_nyc_tracts = 'data/processed/nyc_tracts.gpkg'
path_model_features = 'data/processed/master_features.csv'
path_model_targets = 'data/processed/outcomes_health.csv'
path_performance_metrics = 'data/processed/model_performance_health.csv'
path_final_model_pkl = 'models/xgb_health_model.pkl'
path_output_tract_preds = 'data/processed/predictions_health.csv'
output_gpkg_layer = 'health_predictions'

# ensure cwd is project root for file paths to function properly
project_root = Path(os.getcwd())            # get current directory
while not (project_root / "data").exists(): # keep moving up until in parent
    project_root = project_root.parent
os.chdir(project_root)                      # switch to parent directory

### 1. Load Data

In [2]:
# nyc tracts
gdf_nyc_tracts = gpd.read_file(path_nyc_tracts, layer="tracts")

# features (X), targets (y)
X = pd.read_csv(path_model_features)
y = pd.read_csv(path_model_targets)

# keep copies for accounting and assertin statements
X_original = X.copy()
y_original = y.copy()

### 2. Prepare Data

For this first pass, we'll be focusing on just one target to get the pipeline up and running

In [3]:
TARGET = "OBESITY_CrudePrev"

y = y[['GEOID', TARGET]]

In [4]:
# take note of feature columns
feature_cols = X.columns.tolist()

# get features and targets together for dropping rows without target value for train/eval
df_model = y.merge(X, how="left", on="GEOID")

# TODO: TEMPORARY: DROPPING NAN FEATURE ROWS UNTIL IMPUTATION METHOD DETERMINED
df_model = df_model.dropna(subset=feature_cols)

# for final model predictions, keep all tracts
df_model_all_tracts = df_model.copy()

# for train/eval, select only tracts with a value for the target
df_model = df_model[df_model[TARGET].notna()]

# re-seperate features and target for modeling
X = df_model[feature_cols].copy()
y = df_model[TARGET].copy()

In [5]:
X.head()

,GEOID,distance_to_park_m,park_area_500m_centroid,park_area_1km_centroid,percent_tree_canopy,median_household_income,poverty_rate,unemployment_rate,gini_index,pct_higher_ed,pct_renters,median_gross_rent,pct_rent_burdened,pct_no_vehicle,pop_density_sq_km,pct_age_65_plus
0,36085024402,169.509962,1.206043e+05,6.859291e+05,0.031706,117981.0,0.029496,0.024523,0.4031,0.427270,0.181553,1429.0,0.324759,0.042032,2692.772684,0.193930
1,36085027705,2129.879397,0.000000e+00,0.000000e+00,0.000000,96684.0,0.099770,0.030645,0.4349,0.311325,0.182136,1799.0,0.698630,0.005988,11465.037656,0.191336
2,36085012806,0.000000,1.370448e+06,4.764153e+06,0.000000,61378.0,0.083870,0.037376,0.4349,0.259306,0.648697,1797.0,0.694598,0.004942,4022.827347,0.186134
3,36047024400,622.327717,0.000000e+00,3.497273e+05,0.000000,67500.0,0.398833,0.069648,0.3851,0.313904,0.621649,1748.0,0.601990,0.106186,23808.910470,0.078171
4,36047023000,972.144627,0.000000e+00,1.453035e+03,0.000000,51250.0,0.451197,0.177823,0.5221,0.188471,0.733522,1630.0,0.593068,0.104520,32376.888983,0.078985


In [6]:
y.head()

0    0.284
1    0.281
2    0.328
3    0.242
4    0.290
Name: OBESITY_CrudePrev, dtype: float64

#### Spatial CV - Borough

Here we'll bring in borough so that we can effectively split the data for spatial cross-validation. It's important to split the data by some meaningful geographic grouping because the tracts are not independent of one another: neighboring tracts share characteristics like socioeconomic conditions, health outcomes, air quality, and green coverage. If we split the spatial data at random, we may end up with the scenario where one tract is used for training, and its neighbor is used in testing. This will lead to overestimated model performance because those two tracts are too similar, meaning the model isn't learning to generalize to new places, but rather memorizing information about similar tract contexts/characteristics.

In [7]:
# ensure matching dtypes before join
X.GEOID = X.GEOID.astype(int)
gdf_nyc_tracts.GEOID = gdf_nyc_tracts.GEOID.astype(int)

# get borough names for tracts 
X = X.merge(gdf_nyc_tracts[['GEOID', 'BoroCode']], 
           how='left', 
           on='GEOID',
)

In [8]:
# create borough-based CV groups
groups = X['BoroCode']
cv = GroupKFold(n_splits=5)

In [9]:
# drop identifier and spatial label before modeling
X.drop(columns=['GEOID', 'BoroCode'], inplace=True)

#### Handling Missing Values

TODO: for now, nan rows are dropped (above) until a more comprehensive method is devloped

### 3. Baseline: Linear Regression

Using the folds generated in the Borough-based Spatial CV above, we will train a baseline model (linear regression) and tree-based model (XGBoost) and compare their performance using RMSE and R2

In [10]:
# initialize models (currently fixed hyperparamters for xgb)

# linear regression
lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# xgboost
xgb = XGBRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    random_state=42
)

In [11]:
# run cv training and evaluation loop

scores = {
    "linreg": {"r2":[], "rmse":[]},
    "xgb": {"r2":[], "rmse":[]}
}

for fold, (train_idx, test_idx) in enumerate(cv.split(X, y, groups)):

    print(f"Beginning training and eval for fold {fold}")

    # intialize training and validation sets for this fold
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # -------- Linear Regression --------------
    lr_pipe.fit(X_train, y_train)
    y_pred_lr = lr_pipe.predict(X_test)

    lr_r2_score = r2_score(y_test, y_pred_lr)
    lr_rmse_score = root_mean_squared_error(y_test, y_pred_lr)

    scores["linreg"]["r2"].append(lr_r2_score)
    scores["linreg"]["rmse"].append(lr_rmse_score)

    # ------------- XGBoost -------------------
    xgb.fit(X_train, y_train)
    y_pred_xgb = xgb.predict(X_test)

    xgb_r2_score = r2_score(y_test, y_pred_xgb)
    xgb_rmse_score = root_mean_squared_error(y_test, y_pred_xgb)

    scores["xgb"]["r2"].append(xgb_r2_score)
    scores["xgb"]["rmse"].append(xgb_rmse_score)

    print(f"Completed successfully!")
    
print("Training and validation completed successfully!")

Beginning training and eval for fold 0
Completed successfully!
Beginning training and eval for fold 1
Completed successfully!
Beginning training and eval for fold 2
Completed successfully!
Beginning training and eval for fold 3
Completed successfully!
Beginning training and eval for fold 4
Completed successfully!
Training and validation completed successfully!


### 4. Performance Comparison

In [12]:
mean_scores = []

for model in scores:

    # calculate mean scores across cv folds
    mean_r2 = np.mean(scores[model]["r2"])
    mean_rmse = np.mean(scores[model]["rmse"])

    # print scores for review
    print(f"\n{model} performance evaluation:")
    print(f"R2: ", mean_r2)
    print(f"RMSE: ", mean_rmse)

    # for outputting scores to artifact
    mean_scores.append({"model":model, "r2_cv":mean_r2, "rmse_cv": mean_rmse})


linreg performance evaluation:
R2:  -0.0006778787389890795
RMSE:  0.04741211677282624

xgb performance evaluation:
R2:  -0.07507176441006043
RMSE:  0.04972890684136988


A look at scores for each spatial cv split (i.e. borough)

In [13]:
scores

{'linreg': {'r2': [0.2738823537496753,
   0.2835509878439154,
   -0.23032621535094555,
   0.5718798796823622,
   -0.9023763996199528],
  'rmse': [0.04386251859228618,
   0.04082981215570412,
   0.055272243696570626,
   0.033351415273559025,
   0.06374459414601122]},
 'xgb': {'r2': [0.24330780732246693,
   0.26060642212891194,
   -0.10426174512714037,
   0.24117722499452776,
   -1.0161885313690684],
  'rmse': [0.04477645388708964,
   0.041478455337266795,
   0.052364018682955725,
   0.04440190744240513,
   0.06562369885713208]}}

### 5. Model Summary

**- Explainability -**


**- Prediction Performance -**


### 6. Final Model Predictions

In [14]:
final_model = xgb.fit(X, y)

In [15]:
X.head()

,distance_to_park_m,park_area_500m_centroid,park_area_1km_centroid,percent_tree_canopy,median_household_income,poverty_rate,unemployment_rate,gini_index,pct_higher_ed,pct_renters,median_gross_rent,pct_rent_burdened,pct_no_vehicle,pop_density_sq_km,pct_age_65_plus
0,169.509962,1.206043e+05,6.859291e+05,0.031706,117981.0,0.029496,0.024523,0.4031,0.427270,0.181553,1429.0,0.324759,0.042032,2692.772684,0.193930
1,2129.879397,0.000000e+00,0.000000e+00,0.000000,96684.0,0.099770,0.030645,0.4349,0.311325,0.182136,1799.0,0.698630,0.005988,11465.037656,0.191336
2,0.000000,1.370448e+06,4.764153e+06,0.000000,61378.0,0.083870,0.037376,0.4349,0.259306,0.648697,1797.0,0.694598,0.004942,4022.827347,0.186134
3,622.327717,0.000000e+00,3.497273e+05,0.000000,67500.0,0.398833,0.069648,0.3851,0.313904,0.621649,1748.0,0.601990,0.106186,23808.910470,0.078171
4,972.144627,0.000000e+00,1.453035e+03,0.000000,51250.0,0.451197,0.177823,0.5221,0.188471,0.733522,1630.0,0.593068,0.104520,32376.888983,0.078985


In [16]:
# make sure to use all tracts (originally, we dropped tracts from X, y that had no value for target y. We want those back for final predictions)
X_final = df_model_all_tracts[feature_cols].copy()
y_final = df_model_all_tracts[TARGET].copy()

# set aside GEOID for predictions 
X_geoids = X_final.GEOID
X_final.drop(columns=['GEOID'], inplace=True)

Make sure all tracts are present for model predictions

In [17]:
# this will fail while imputation not set up (since I am merely dropping tract rows with no feature values)
try:
    assert(len(X_final) == len(gdf_nyc_tracts))
    print("All tracts accounted for")

# if this passes, then above only failed because of lack of impuation handling
# if this fails, then tracts are inexplicably unaccounted for, and attention is needed
except:
    assert(len(X_final) == (len(gdf_nyc_tracts) - X_original[feature_cols].isna().any(axis=1).sum()))
    print("All tracts accounted for, imputation-dropped tracts not included in predictions")


All tracts accounted for, imputation-dropped tracts not included in predictions


In [18]:
X_final.head()

,distance_to_park_m,park_area_500m_centroid,park_area_1km_centroid,percent_tree_canopy,median_household_income,poverty_rate,unemployment_rate,gini_index,pct_higher_ed,pct_renters,median_gross_rent,pct_rent_burdened,pct_no_vehicle,pop_density_sq_km,pct_age_65_plus
0,169.509962,1.206043e+05,6.859291e+05,0.031706,117981.0,0.029496,0.024523,0.4031,0.427270,0.181553,1429.0,0.324759,0.042032,2692.772684,0.193930
1,2129.879397,0.000000e+00,0.000000e+00,0.000000,96684.0,0.099770,0.030645,0.4349,0.311325,0.182136,1799.0,0.698630,0.005988,11465.037656,0.191336
2,0.000000,1.370448e+06,4.764153e+06,0.000000,61378.0,0.083870,0.037376,0.4349,0.259306,0.648697,1797.0,0.694598,0.004942,4022.827347,0.186134
3,622.327717,0.000000e+00,3.497273e+05,0.000000,67500.0,0.398833,0.069648,0.3851,0.313904,0.621649,1748.0,0.601990,0.106186,23808.910470,0.078171
4,972.144627,0.000000e+00,1.453035e+03,0.000000,51250.0,0.451197,0.177823,0.5221,0.188471,0.733522,1630.0,0.593068,0.104520,32376.888983,0.078985


In [19]:
tract_preds = final_model.predict(X_final)

In [20]:
df_tract_preds = pd.DataFrame({
    "GEOID": X_geoids,
    "actual": y_final,
    "predicted": tract_preds,
    "residual": y_final - tract_preds
})

In [21]:
df_tract_preds.head()

,GEOID,actual,predicted,residual
0,36085024402,0.284,0.281582,0.002418
1,36085027705,0.281,0.276005,0.004995
2,36085012806,0.328,0.317195,0.010805
3,36047024400,0.242,0.259259,-0.017259
4,36047023000,0.290,0.295527,-0.005527


### 7. Output Modeling Artifacts

A. Performance Metrics

In [22]:
df_perform_metrics = pd.DataFrame(mean_scores)

In [23]:
df_perform_metrics

,model,r2_cv,rmse_cv
0,linreg,-0.000678,0.047412
1,xgb,-0.075072,0.049729


In [24]:
df_perform_metrics.to_csv(path_performance_metrics, index=False)

B. Final Model

In [25]:
joblib.dump(final_model, path_final_model_pkl)

['models/xgb_health_model.pkl']

C. Tract-Level Predictions

In [26]:
# save to csv
df_tract_preds.to_csv(path_output_tract_preds ,index=False)

In [27]:
# and additionally save as layer in nyc tracts geopackage
gdf_nyc_tracts.GEOID = gdf_nyc_tracts.GEOID.astype(int)
df_tract_preds.GEOID = df_tract_preds.GEOID.astype(int)

gdf_tracts_preds = gdf_nyc_tracts.merge(
    df_tract_preds,
    on='GEOID',
    how='left'
)
gdf_tracts_preds.to_file(path_nyc_tracts, layer=output_gpkg_layer)